# Django 初始化

In [1]:
import django_setup

Development settings loaded
INSTALLED_APPS: ['django.contrib.admin', 'django.contrib.auth', 'django.contrib.contenttypes', 'django.contrib.sessions', 'django.contrib.messages', 'django.contrib.staticfiles', 'rest_framework', 'corsheaders', 'storages', 'apps.authentication', 'apps.files', 'apps.projects', 'apps.doc_analysis', 'apps.chat', 'django_filters', 'drf_spectacular', 'rest_framework_simplejwt.token_blacklist', 'django_celery_results', 'django_celery_beat']


INFO 2025-03-07 13:15:01,293 storage default_storage 的类型: COSStorage


Settings从哪里加载？: config.settings.development
项目根目录对么？: C:\Users\huiwa\Documents\_All_Projects\BidPilot_new\backend
文件存储settings对么？: apps.files.storage.COSStorage
文件default_storage对么？: COSStorage

已经安装的应用 Installed Apps 完整了么？:
- django.contrib.admin
- django.contrib.auth
- django.contrib.contenttypes
- django.contrib.sessions
- django.contrib.messages
- django.contrib.staticfiles
- rest_framework
- corsheaders
- storages
- apps.authentication
- apps.files
- apps.projects
- apps.doc_analysis
- apps.chat
- django_filters
- drf_spectacular
- rest_framework_simplejwt.token_blacklist
- django_celery_results
- django_celery_beat


In [2]:
#导入所有模型
from django.contrib.auth import get_user_model
from apps.files.models import FileRecord
from apps.projects.models import Project, ProjectHistory
from apps.doc_analysis.models import DocumentAnalysis

User = get_user_model()
projects = Project.objects.all()  #事实上可以直接使用
files = FileRecord.objects.all()
analyses = DocumentAnalysis.objects.all()


数据库数据清理，只保留18501771516用户的数据


In [ ]:
# 删除18501771516用户以外的项目
deleted_count = Project.objects.exclude(creator__phone='18501771516').delete()
print(f"已删除 {deleted_count[0]} 个其他项目")


# 删除18501771516用户以外的文件
deleted_count = FileRecord.objects.exclude(owner__phone='18501771516').delete()
print(f"已删除 {deleted_count[0]} 个其他文件")

# 删除18501771516用户以外的分析
deleted_count = DocumentAnalysis.objects.exclude(created_by__phone='18501771516').delete()
print(f"已删除 {deleted_count[0]} 个其他分析")

# 删除18501771516用户以外的用户
deleted_count = User.objects.exclude(phone='18501771516').delete()
print(f"已删除 {deleted_count[0]} 个其他用户")

# <font color="red">Part 1 - 用户管理</font>


In [ ]:
# 用户模型所有字段
print("User")
print("-----------常规字段--------------")
for field in User._meta.fields:
    print(f"- {field.name}: {field.__class__.__name__}")

print("\n-----------反向关系字段-------------")
# 反向关系字段
for field in User._meta.get_fields():
    # 检查是否是反向关系字段
    if field.is_relation and field.auto_created:
        related_name = field.related_name or f"{field.name}_set"
        print(f"- {related_name}: {field.__class__.__name__} (来自 {field.related_model.__name__})")

print("\n-----------多对多字段-------------")
for field in User._meta.many_to_many:
    print(f"- {field.name}: {field.__class__.__name__}")



### 打印所有用户

In [ ]:
# 打印用户列表
users = User.objects.all()
for user in users:
    print(f"用户：{user.phone}," 
          f"项目数：{user.created_projects.all().count()}," 
          f"文件数：{user.owned_files.all().count()}",
          f"分析数：{user.created_analyses.all().count()}"
          )


### 过滤用户

In [ ]:
# 打印用户的所有字段值
for user in User.objects.filter(phone='18501771516'):
    print(f"用户ID: {user.id}, 用户名: {user.username}")
    # 打印用户的所有字段值

    for field in User._meta.fields:
        print(f"  {field.name}: {getattr(user, field.name)}")
    print("---")



In [ ]:
# 删除selected之外的所有用户
selected_phones = ['18501771516', '13800138000']
deleted_count = User.objects.exclude(phone__in=selected_phones).delete()
print(f"已删除 {deleted_count[0]} 个其他用户")


In [8]:
# 清空所有上传的文件
#delete_count = User.objects.all().delete()
#print(delete_count[0])

# <font color="red">Part 2 - 项目管理</font>


In [ ]:
# 项目模型所有字段
print("Project")
print("-----------常规字段--------------")
for field in Project._meta.fields:
    print(f"- {field.name}: {field.__class__.__name__}")

print("\n-----------反向关系字段-------------")
# 反向关系字段
for field in Project._meta.get_fields():
    # 检查是否是反向关系字段
    if field.is_relation and field.auto_created:
        related_name = field.related_name or f"{field.name}_set"
        print(f"- {related_name}: {field.__class__.__name__} (来自 {field.related_model.__name__})")

print("\n-----------多对多字段-------------")
for field in Project._meta.many_to_many:
    print(f"- {field.name}: {field.__class__.__name__}")

In [ ]:
# 打印项目列表
projects = Project.objects.all()
for project in projects:
    print(f"项目编号:{project.project_code}",
          f"项目名：{project.project_name}," 
          f"创建者：{project.creator.phone}," 
          f"关联分析数：{project.document_analyses.all().count()}"
          )

In [11]:
# 清空所有上传的文件
#delete_count = Project.objects.all().delete()
#print(delete_count[0])

In [ ]:
# 打印特定用户的所有项目的所有字段值
for project in Project.objects.filter(creator__phone='18501771516',project_code='BP-2025-WELFARE-0001'):
    print(f"项目ID: {project.id}, 项目名: {project.project_name}")
    # 打印项目的所有字段值

    for field in project._meta.fields:
        print(f"  {field.name}: {getattr(project, field.name)}")
    print("---")



# <font color="red">Part 3 - 文件管理</font>


In [ ]:
# 文件所有字段
# 文件的所有常规字段
print("FileRecord")
print("-----------常规字段--------------")
for field in FileRecord._meta.fields:
    print(f"- {field.name}: {field.__class__.__name__}")

print("\n-----------反向关系字段-------------")
# 反向关系字段
for field in FileRecord._meta.get_fields():
    # 检查是否是反向关系字段
    if field.is_relation and field.auto_created:
        related_name = field.related_name or f"{field.name}_set"

        print(f"- {related_name}: {field.__class__.__name__} (来自 {field.related_model.__name__})")

print("\n-----------多对多字段-------------")
for field in FileRecord._meta.many_to_many:
    print(f"- {field.name}: {field.__class__.__name__}")

In [ ]:
# 打印项目列表
files = FileRecord.objects.all()
for file in files:
    print(f"文件ID:{file.id}",
          f"文件类型:{file.type}",
          f"文件名：{file.name}," 
          f"拥有者：{file.owner.phone}," 
          f"关联分析数：{file.document_analyses.all().count()}"
          )

In [18]:
# 清空所有上传的文件
#delete_count = FileRecord.objects.all().delete()
#print(delete_count[0])

In [19]:
# 打印特定用户的所有项目的所有字段值
for file in FileRecord.objects.filter(owner__phone='18501771516',name='Test 文本.docx'):
    print(f"文件ID: {file.id}, 文件名: {file.name}")
    # 打印文件的所有字段值


    for field in file._meta.fields:
        print(f"  {field.name}: {getattr(file, field.name)}")
    print("---")

<font color="red">将文件关联到项目</font>


In [17]:
#delete_count = FileProjectLink.objects.filter(project=project).delete()
#print(deleted_count[0])

# <font color=red>Part 4 - 分析管理 </font>

In [ ]:
# 文档分析的所有字段
# 常规字段
print("DocumentAnalysis")
print("-----------常规字段--------------")
for field in DocumentAnalysis._meta.fields:
    print(f"- {field.name}: {field.__class__.__name__}")

print("\n-----------反向关系字段-------------")
# 反向关系字段
for field in DocumentAnalysis._meta.get_fields():
    # 检查是否是反向关系字段
    if field.is_relation and field.auto_created:
        related_name = field.related_name or f"{field.name}_set"

        print(f"- {related_name}: {field.__class__.__name__} (来自 {field.related_model.__name__})")

print("\n-----------多对多字段-------------")
for field in DocumentAnalysis._meta.many_to_many:
    print(f"- {field.name}: {field.__class__.__name__}")

In [14]:
analyses = DocumentAnalysis.objects.all()

In [ ]:
print(analyses)

In [16]:
# 打印项目列表
analyses = DocumentAnalysis.objects.all()
for analysis in analyses:
    print(f"分析号:{analysis.id}\n")

In [22]:
# 打印项目列表
analyses = DocumentAnalysis.objects.all()
for analysis in analyses:
    print(f"分析号:{analysis.id}\n",
          f"分析名称：{analysis.title}\n",
          f"分析所在项目：{analysis.project.project_name}\n", 
          f"分析的文件：{analysis.file_record.name}\n" ,
         # f"分析的阶段：{analysis.status}\n",
         # f"分析的问题：{analysis.analysis_questions}\n",
         # f"分析结果：{analysis.analysis_result}\n",
         # f"分析创建者：{analysis.created_by.phone}\n",
         # f"分析用时：{analysis.processing_time}\n",
          )

In [ ]:
delete_count = DocumentAnalysis.objects.all().delete()
print(delete_count[0])